### Alternating Least Squares


ALS (Alternating Least Squares) is an implicit recommendation algorithm to make a recommendation of products and product categories to the users. ALS is an iterative optimization process where for every iteration it tries to arrive closer and closer to a factorized representation of the original data.

In [1]:
import pandas as pd
import os
import numpy as np 
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler


os.chdir(r'N:\Digital Marketing Analytics')

In [18]:
Olist_db = pd.read_csv('Olist_Recommendation_Dataset.csv')

In [19]:
Olist_db.columns

Index(['Customer State', 'Review Score', 'Purchase Timestamp', 'Purchase Date',
       'Monetary', 'Customer Segment', 'Marketing Action', 'Customer_ID',
       'Order_ID', 'Product_ID', 'product_name', 'aisle', 'department',
       'Product Category'],
      dtype='object')

In [20]:
Olist_db

,Customer State,Review Score,Purchase Timestamp,Purchase Date,Monetary,Customer Segment,Marketing Action,Customer_ID,Order_ID,Product_ID,product_name,aisle,department,Product Category
0,RJ,5,2017-09-13 08:59:02,2017-09-13 00:00:00,281.30,Potential Loyalists,Cross Sell Recommendations and Discount coupons,15438,0,8446,Utility Lighter,more household,household,more household
1,GO,5,2017-06-28 11:52:20,2017-06-28 00:00:00,73.86,Lost Customers,Don't spend too much trying to re-acquire,22529,7229,8446,Utility Lighter,more household,household,more household
2,MG,4,2018-05-18 10:25:53,2018-05-18 00:00:00,108.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,17139,31508,8446,Utility Lighter,more household,household,more household
3,PR,5,2017-08-01 18:38:42,2017-08-01 00:00:00,472.06,Loyal Customers,Loyality programs;Cross Sell,33008,42252,8446,Utility Lighter,more household,household,more household
4,MG,5,2017-08-10 21:48:40,2017-08-10 00:00:00,1415.74,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,41377,47321,8446,Utility Lighter,more household,household,more household
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100846,RJ,1,2017-12-09 13:51:24,2017-12-09 00:00:00,618.88,Potential Loyalists,Cross Sell Recommendations and Discount coupons,3664,94603,14949,Hand Luggage,missing,missing,Hand Luggage
100847,SP,4,2017-07-13 11:24:43,2017-07-13 00:00:00,157.30,Hibernating - Almost Lost,Aggressive price incentives,10847,87800,4042,Chicken Curry with Seasoned Basmati Rice,frozen meals,frozen,frozen meals
100848,SP,3,2018-07-27 10:44:02,2018-07-27 00:00:00,68.35,Potential Loyalists,Cross Sell Recommendations and Discount coupons,32799,88387,8881,Apple Macbook,ice cream ice,frozen,Apple Macbook
100849,SC,5,2018-08-21 11:29:05,2018-08-21 00:00:00,314.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,40805,90090,13612,Perfumes,candy chocolate,snacks,Perfumes


In [21]:
features = ['Customer_ID', 'product_name', 'Product_ID', 'Review Score']
Olist_db_ = Olist_db[features]
Olist_db_ = Olist_db_.rename(columns={'Review Score': 'Ratings'})
Olist_db_ = Olist_db_.drop_duplicates()

In [22]:
from scipy.sparse import csr_matrix

In [23]:
Olist_db_ = Olist_db_[:20000]

In [24]:
Olist_db_.Customer_ID.unique

<bound method Series.unique of 0        15438
1        22529
2        17139
3        33008
4        41377
         ...  
20004    36884
20005    38743
20006    23423
20007    25440
20008    26766
Name: Customer_ID, Length: 20000, dtype: int64>

In [25]:
# pivot ratings into movie features
df_product_features = Olist_db_.reset_index().pivot_table(
    index='Customer_ID',
    columns='Product_ID',
    values='Ratings'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_product_features = csr_matrix(df_product_features.values)

In [26]:
df_product_features.head()

Product_ID,63,69,151,161,185,219,226,238,255,311,...,32021,32050,32052,32083,32115,32134,32164,32170,32171,32203
Customer_ID,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_product_features.shape

(15534, 1172)

In [28]:
X = df_product_features.T
X.head()

Customer_ID,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
Product_ID,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
X.index[:-20]

Int64Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,
              311,
            ...
            31567, 31575, 31585, 31605, 31607, 31631, 31654, 31668, 31671,
            31692],
           dtype='int64', name='Product_ID', length=1152)

In [29]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1172, 10)

In [30]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1172, 1172)

In [31]:
correlation_matrix

array([[ 1.        , -0.34654158,  0.40575627, ...,  0.5257649 ,
         0.34932972,  0.70872076],
       [-0.34654158,  1.        ,  0.59444482, ..., -0.07122444,
         0.15293433, -0.20599487],
       [ 0.40575627,  0.59444482,  1.        , ...,  0.34763895,
         0.25205425,  0.31837942],
       ...,
       [ 0.5257649 , -0.07122444,  0.34763895, ...,  1.        ,
         0.45381189,  0.29968869],
       [ 0.34932972,  0.15293433,  0.25205425, ...,  0.45381189,
         1.        , -0.09604217],
       [ 0.70872076, -0.20599487,  0.31837942, ...,  0.29968869,
        -0.09604217,  1.        ]])

In [32]:
fave_prod = Olist_db_.groupby(['Customer_ID']).max()['Product_ID'].to_frame()

In [33]:
fave_prod = fave_prod.reset_index()
fave_prod

,Customer_ID,Product_ID
0,5,10630
1,9,6927
2,12,15675
3,15,27919
4,16,10630
...,...,...
15529,41419,13331
15530,41421,18573
15531,41422,15250
15532,41424,21641


In [74]:
def get_prod_id(customer_id):
    
    product_id = fave_prod[fave_prod.Customer_ID == customer_id]['Product_ID']
    return product_id
    
    
prd_id = get_prod_id(41422)
Product_id = prd_id.iloc[0]
Product_id

15250

In [75]:
product_names = list(X.index)
product_ID_Idx = product_names.index(Product_id)
product_ID_Idx

531

In [76]:
correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID.shape

(1172,)

### Recommending top 10 highly coorelated products in sequence

In [79]:
Recommend = list(X.index[correlation_product_ID > 0.70])
# Removes the item already bought by the customer
Recommend.remove(Product_id) 
Recommend[0:20]

[819,
 1477,
 3196,
 3417,
 5732,
 6122,
 12089,
 12626,
 15807,
 16929,
 16961,
 16963,
 23263,
 23302,
 23479,
 32083,
 32171]

In [52]:
## Getting Product names froom prediction 

predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['Product_ID']
predictions

,Product_ID
0,219
1,238
2,4053
3,6099
4,6829
5,8558
6,8710
7,9159
8,11866
9,13108


In [53]:
predictions['Product Name'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.Product_ID == x]['product_name'].unique()[0])

In [54]:
predictions[:10]

,Product_ID,Product Name
0,219,Snakku
1,238,Alpha-Bits Multigrain Cereal
2,4053,Fancy Feast Appetizers White Meat Chicken & Fl...
3,6099,Keurig Nantucket Blend Medium Roast Coffee K Cups
4,6829,Quest Bar Chocolate Chip Cookie Dough Protein Bar
5,8558,"Cottage Cheese, Natural Small Curd, 4% Milkfat..."
6,8710,Protein
7,9159,Chopped Ginger
8,11866,Happy Birthday Candle Pick Set
9,13108,"Vegetable Oil, Pure & 100% Natural"


In [55]:
Olist_db_.to_csv('Olist_db_ALS.csv', index=False)

In [56]:
X.to_csv('X_ALS.csv')

In [57]:
correlation_matrix

array([[ 1.        , -0.34654158,  0.40575627, ...,  0.5257649 ,
         0.34932972,  0.70872076],
       [-0.34654158,  1.        ,  0.59444482, ..., -0.07122444,
         0.15293433, -0.20599487],
       [ 0.40575627,  0.59444482,  1.        , ...,  0.34763895,
         0.25205425,  0.31837942],
       ...,
       [ 0.5257649 , -0.07122444,  0.34763895, ...,  1.        ,
         0.45381189,  0.29968869],
       [ 0.34932972,  0.15293433,  0.25205425, ...,  0.45381189,
         1.        , -0.09604217],
       [ 0.70872076, -0.20599487,  0.31837942, ...,  0.29968869,
        -0.09604217,  1.        ]])

In [48]:
correlation_matrix.shape

(3638, 3638)

In [58]:
import json

with open('correlation_matrix_ALS.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

### Prediction Pipeline 

In [47]:
### use 3 files to predict for a User

In [59]:
Olist_db_ = pd.read_csv('Olist_db_ALS.csv')

In [66]:
X = pd.read_csv('X_ALS.csv', index_col=0)
X

,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
Product_ID,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
X.columns[4000:4020]

Index(['10670', '10671', '10675', '10677', '10679', '10680', '10681', '10682',
       '10685', '10686', '10689', '10694', '10696', '10698', '10699', '10700',
       '10703', '10712', '10713', '10714'],
      dtype='object')

In [61]:
import json
with open('correlation_matrix_ALS.txt') as f:
    correlation_matrix = json.load(f)

In [62]:
correlation_matrix = np.array(correlation_matrix)

In [67]:
X.index

Int64Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,
              311,
            ...
            32021, 32050, 32052, 32083, 32115, 32134, 32164, 32170, 32171,
            32203],
           dtype='int64', name='Product_ID', length=1172)

### Streamlit

In [80]:
#Olist_db_ = pd.read_csv('Olist_db_ALS.csv')
#X = pd.read_csv('X_ALS.csv')
    
#with open('correlation_matrix_ALS.txt') as f:
#    correlation_matrix = json.load(f)

#correlation_matrix = np.array(correlation_matrix)

def Recommendations_ALS(Customer_id):
    #Olist_db_ = Olist_db_[Olist_db_.Customer_ID < 3600]
    fave_prod = Olist_db_.groupby(['Customer_ID']).max()['Product_ID'].to_frame()
    fave_prod = fave_prod.reset_index()
    
    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.Customer_ID == Customer_id]['Product_ID']
    Product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(Product_id)
    
    correlation_product_ID = correlation_matrix[product_ID_Idx]
    
    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(Product_id) 
    
    ## Getting Product names froom prediction 

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['Product_ID']

    predictions['Product Name'] = predictions.Product_ID.apply(lambda x : Olist_db_[Olist_db_.Product_ID == x]['product_name'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations



In [91]:
Recommendations = Recommendations_ALS(10679)
Recommendations[:10]

,Product_ID,Product Name
0,185,Chicken Nugget Meal
1,504,Velveeta Shells & Cheese Bold Jalapeno
2,2694,Superberry Kombucha
3,2767,Hand Luggage
4,3947,Apple Macbook
5,4575,"Beef Jerky, Teriyaki"
6,4621,Organic Granny Smith Apple Bag
7,5140,Lean & Fit Coconut Lemongrass Chicken
8,6367,Sugar Free Concord Grape Jam
9,6488,Grape Water Enhancer
